In [4]:
!pip install pymdptoolbox
import mdptoolbox.example
import numpy as np
import pandas as pd
import itertools
from collections import defaultdict
from collections import Counter
import random
import timeit
import random
import time


np.random.seed(123)



datap = pd.read_csv('final_labeled_data_v2.csv', sep=',', header=0)
sas=pd.DataFrame({'count' : datap.groupby( [ 's', 'a','sp'] ).size()}).reset_index()
sa=pd.DataFrame({'count' : datap.groupby( [ 's', 'a'] ).size()}).reset_index()



states=14
actions=14
p = [1 for a in range(0,states)]
a=list(range(1, actions+1))
s=list(range(1, states+1))

ar_count=pd.DataFrame({'count' : datap.groupby( [ 'a', 'r'] ).size()}).reset_index()
a_count=pd.DataFrame({'count' : datap.groupby( [ 'a'] ).size()}).reset_index()

r= pd.merge(ar_count, a_count,  how='left', left_on=['a'], right_on = ['a'])

r.columns = ['a', 'r','ar_count','a_count']

r['r_prob'] = r['ar_count']/r['a_count']

pos_r=datap.loc[(datap['r'] > 0)]



gamma=0.95

from sklearn.model_selection import train_test_split
#predict R
x = datap.drop('r', axis=1)
x = x.drop('sp', axis=1)
y = datap['r']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

from sklearn.tree import DecisionTreeClassifier  
classifier = DecisionTreeClassifier(class_weight='balanced')  
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)  

from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_test, y_pred))  
print(classification_report(y_test, y_pred))



from sklearn.utils.extmath import cartesian
ra=cartesian([s, a])
y_pred = classifier.predict(ra)

r=np.column_stack((ra, y_pred))

#print(r)
r_pd = pd.DataFrame(r)
#t_pd.groupby(0)[0].mean()
r_pd.columns = ['s', 'a','r_pred']
print(r_pd.columns)
r_f=r_pd[['s','r_pred']].groupby([ 's']).max()

len(list(r_f['r_pred']))


inds = r_pd.groupby(['s'])['r_pred'].transform(max) == r_pd['r_pred']
r_pd = r_pd[inds]
r_pd.reset_index(drop=True, inplace=True)

r_i=r_pd[['s','a']].groupby([ 's']).max()

#print(list(r_pd['a']))
#r_pd.to_csv('p_regres.csv')


You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


[[12103  4390  2036  1451  2159  2643]
 [   21    30    19    14    17    16]
 [   39    58    21    32    36    22]
 [   77   126    80    48    84    70]
 [  107   143   101    60   132    98]
 [   63    87    63    44    81   113]]
             precision    recall  f1-score   support

         -5       0.98      0.49      0.65     24782
          1       0.01      0.26      0.01       117
          2       0.01      0.10      0.02       208
          3       0.03      0.10      0.04       485
          4       0.05      0.21      0.08       641
          5       0.04      0.25      0.07       451

avg / total       0.91      0.47      0.61     26684

Index(['s', 'a', 'r_pred'], dtype='object')


In [6]:
#### random forest for T
# Use numpy to convert to arrays
import numpy as np
# Labels are the values we want to predict
x = datap.drop('r', axis=1)
x = x.drop('sp', axis=1)
y = datap['sp']

labels = np.array(y)
# Remove the labels from the features
# axis 1 refers to the columns
features= x
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels)

# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

predictions = rf.predict(ra)
t=np.column_stack((ra, predictions))
t=np.rint(t)

#print(t)


Mean Absolute Error: 2.85 degrees.


In [21]:
U=list(r_f['r_pred']) #initialize value function to average reward at each position
p=(list(r_i['a']))



#now all I need is table of s rows, a columns, with values as predicted s'.  From there, I'll create a 
#dirichlet centered at s' for each cell
t_df = pd.DataFrame(t)
t_df.columns = ['s', 'a','sp']

t_piv=t_df.pivot(index='s', columns='a', values='sp')
t_num=t_piv.values
#print(t_num)


r_df = pd.DataFrame(r)
r_df.columns = ['s', 'a','r']
r_piv=r_df.pivot(index='s', columns='a', values='r')
r_num=r_piv.values
#print(r_num)


#np.savetxt("t_num.csv", t_num, delimiter=",")


#now all I need is table of s rows, a columns, with values as predicted s'.  From there, I'll create a 
#dirichlet centered at s' for each cell
U=list(r_f['r_pred']) #initialize value function to average reward at each position
u_matrix=np.zeros(shape=(states,actions))

p=(list(r_i['a']))
gamma=0.8

t_df = pd.DataFrame(t)
t_df.columns = ['s', 'a','sp']

t_piv=t_df.pivot(index='s', columns='a', values='sp')
t_num=t_piv.values
#print(t_num)


r_df = pd.DataFrame(r)
r_df.columns = ['s', 'a','r']
r_piv=r_df.pivot(index='s', columns='a', values='r')
r_num=r_piv.values
#print(r_num)


#np.savetxt("t_num.csv", t_num, delimiter=",")

t0 = time.time()

iterations=30

u_score=sum(U)

for k in range(1,iterations):
    def f(x):
        dirich = [5 for i in range(states)]
        dirich[int(x)-1]= 500
        t_sas = np.random.dirichlet(dirich)
        return gamma*np.dot(t_sas,U)

    vf = np.vectorize(f)

    test=np.vectorize(f)(t_num)
    u_matrix=r_num+test
    U=u_matrix.max(axis=1)
    u_score=U.sum(axis=0)
    print("U_score:",u_score,"P:", (u_matrix.argmax(axis=1))+1)

#print(np.argmax(u_matrix.max(axis=0)))
p=(u_matrix.argmax(axis=1))+1

t1 = time.time()

np.savetxt("p_large_test.csv", p, delimiter=",")
np.savetxt("u_large_test.csv", U, delimiter=",")

#print(t1-t0)
print("Final policy:",p)




U_score: 126.0 P: [ 7  2  1  4 13 13 13  4  1  4 13  4  3 13]
U_score: 170.8 P: [ 7  2  7  4 13 13 13  4  3  4 13  3  3 13]
U_score: 206.64 P: [ 7  2  1 13 13 13 13  4  3  4 13  4  3 13]
U_score: 235.312 P: [ 7 12  1  4 13 13 13  4  1  4 13  3 13 13]
U_score: 258.2496000000001 P: [ 7  2  1  4 13 13 13  4  1  4 13  3  7 13]
U_score: 276.59968000000003 P: [ 7 12  1  4 13 13 13  4  1  4 13  3  3 13]
U_score: 291.27974400000005 P: [ 7 13  3  4 13 13 13  4  1  5 13 13 13 13]
U_score: 303.02379520000005 P: [ 7 12  3 13 13 13 13  4  1 13 13  3  3 13]
U_score: 312.41903616 P: [ 7  3  3  4 13 13 13  4  1  4 13  3 13 13]
U_score: 319.9352289280001 P: [ 7  2 13  4 13 13 13  4  3  5 13  4  7 13]
U_score: 325.94818314239996 P: [ 7 12  1  4 13 13 13  4  1 13 13 13  3 13]
U_score: 330.75854651391995 P: [ 7 12  1  4 13 13 13  4  1  5 13  4  7 13]
U_score: 334.60683721113605 P: [ 7  2 13  4 13 13 13  4  1  4 13  3  3 13]
U_score: 337.6854697689089 P: [ 7 12  3  4 13 13 13  4  1  4 13  7  3 13]
U_score: